<a href="https://colab.research.google.com/github/cjfghk5697/anomaly-detection-competition/blob/main/inference_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd "/content/drive/MyDrive/input/"
#!unzip -q "/content/drive/MyDrive/input/test.zip"

[Errno 2] No such file or directory: '/content/drive/MyDrive/input/'
/content
unzip:  cannot find or open /content/drive/MyDrive/input/test.zip, /content/drive/MyDrive/input/test.zip.zip or /content/drive/MyDrive/input/test.zip.ZIP.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip3 install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 7.6 MB/s 


In [4]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import random
import timm

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time
from typing import Tuple, Sequence, Callable
from PIL import Image
import cv2

device = torch.device('cuda')

In [19]:
test_png = sorted(glob('/content/drive/MyDrive/input/test/*.png'))

In [6]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (512, 512))
    return img

In [20]:
test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|██████████| 2154/2154 [04:07<00:00,  8.70it/s]


In [21]:
transforms_test = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

In [22]:
train_y = pd.read_csv('/content/drive/MyDrive/input/train_df.csv')

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

In [23]:
class Custom_dataset(Dataset):
    def __init__(self, 
                 img_paths, 
                 labels, 
                 mode='train',
                 transforms= Sequence[Callable]
            ) -> None:
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
        self.transforms = transforms

    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            augmentation = random.randint(0,2)
            if augmentation==1:
                img = img[::-1].copy()
            elif augmentation==2:
                img = img[:,::-1].copy()
        if self.mode=='test':
            pass
        img = Image.fromarray(img) # NumPy array to PIL image
        if self.transforms is not None:
            img = self.transforms(img)        
        label = self.labels[idx]
        return img, label
        
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=88)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [24]:
# Test
batch_size = 32

test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test',transforms=transforms_test)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Network().to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/input/test_model.pth'), strict=False)

<All keys matched successfully>

In [26]:
model.eval()
f_pred = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [27]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

In [31]:
submission = pd.read_csv('/content/drive/MyDrive/input/sample_submission.csv')

submission["label"] = f_result

submission
submission.to_csv("/content/drive/MyDrive/input/baseline.csv", index = False)